## Practical 1: word2vec
<p>Oxford CS - Deep NLP 2017<br>
https://www.cs.ox.ac.uk/teaching/courses/2016-2017/dl/</p>
<p>[Yannis Assael, Brendan Shillingford, Chris Dyer]</p>

This practical is presented as an IPython Notebook, with the code written for recent versions of **Python 3**. The code in this practical will not work with Python 2 unless you modify it. If you are using your own Python installation, ensure you have a setup identical to that described in the installation shell script (which is intended for use with the department lab machines). We will be unable to support installation on personal machines due to time constraints, so please use the lab machines and the setup script if you are unfamiliar with how to install Anaconda.

To execute a notebook cell, press `shift-enter`. The return value of the last command will be displayed, if it is not `None`.

Potentially useful library documentation, references, and resources:

* IPython notebooks: <https://ipython.org/ipython-doc/3/notebook/notebook.html#introduction>
* Numpy numerical array library: <https://docs.scipy.org/doc/>
* Gensim's word2vec: <https://radimrehurek.com/gensim/models/word2vec.html>
* Bokeh interactive plots: <http://bokeh.pydata.org/en/latest/> (we provide plotting code here, but click the thumbnails for more examples to copy-paste)
* scikit-learn ML library (aka `sklearn`): <http://scikit-learn.org/stable/documentation.html>
* nltk NLP toolkit: <http://www.nltk.org/>
* tutorial for processing xml in python using `lxml`: <http://lxml.de/tutorial.html> (we did this for you below, but in case you need it in the future)

In [53]:
import numpy as np
import os
from random import shuffle
import re

In [54]:
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

### Part 0: Download the TED dataset

In [56]:
import urllib.request
import zipfile
import lxml.etree

In [57]:
# Download the dataset if it's not already there: this may take a minute as it is 75MB
if not os.path.isfile('ted_en-20160408.zip'):
    urllib.request.urlretrieve("https://wit3.fbk.eu/get.php?path=XML_releases/xml/ted_en-20160408.zip&filename=ted_en-20160408.zip", filename="ted_en-20160408.zip")

In [58]:
# For now, we're only interested in the subtitle text, so let's extract that from the XML:
with zipfile.ZipFile('ted_en-20160408.zip', 'r') as z:
    doc = lxml.etree.parse(z.open('ted_en-20160408.xml', 'r'))
input_text = '\n'.join(doc.xpath('//content/text()'))
del doc

### Part 1: Preprocessing

In this part, we attempt to clean up the raw subtitles a bit, so that we get only sentences. The following substring shows examples of what we're trying to get rid of. Since it's hard to define precisely what we want to get rid of, we'll just use some simple heuristics.

In [59]:
i = input_text.find("Hyowon Gweon: See this?")
input_text[i-20:i+150]

' baby does.\n(Video) Hyowon Gweon: See this? (Ball squeaks) Did you see that? (Ball squeaks) Cool. See this one? (Ball squeaks) Wow.\nLaura Schulz: Told you. (Laughs)\n(Vide'

Let's start by removing all parenthesized strings using a regex:

In [60]:
input_text_noparens = re.sub(r'\([^)]*\)', '', input_text)

We can verify the same location in the text is now clean as follows. We won't worry about the irregular spaces since we'll later split the text into sentences and tokenize it anyway.

In [61]:
i = input_text_noparens.find("Hyowon Gweon: See this?")
input_text_noparens[i-20:i+150]

"hat the baby does.\n Hyowon Gweon: See this?  Did you see that?  Cool. See this one?  Wow.\nLaura Schulz: Told you. \n HG: See this one?  Hey Clara, this one's for you. You "

Now, let's attempt to remove speakers' names that occur at the beginning of a line, by deleting pieces of the form "`<up to 20 characters>:`", as shown in this example. Of course, this is an imperfect heuristic. 

In [62]:
sentences_strings_ted = []
for line in input_text_noparens.split('\n'):
    m = re.match(r'^(?:(?P<precolon>[^:]{,20}):)?(?P<postcolon>.*)$', line)
    sentences_strings_ted.extend(sent for sent in m.groupdict()['postcolon'].split('.') if sent)

# Uncomment if you need to save some RAM: these strings are about 50MB.
# del input_text, input_text_noparens

# Let's view the first few:
sentences_strings_ted[:5]

["Here are two reasons companies fail: they only do more of the same, or they only do what's new",
 'To me the real, real solution to quality growth is figuring out the balance between two activities: exploration and exploitation',
 ' Both are necessary, but it can be too much of a good thing',
 'Consider Facit',
 " I'm actually old enough to remember them"]

Now that we have sentences, we're ready to tokenize each of them into words. This tokenization is imperfect, of course. For instance, how many tokens is "can't", and where/how do we split it? We'll take the simplest naive approach of splitting on spaces. Before splitting, we remove non-alphanumeric characters, such as punctuation. You may want to consider the following question: why do we replace these characters with spaces rather than deleting them? Think of a case where this yields a different answer.

In [63]:
sentences_ted = []
for sent_str in sentences_strings_ted:
    tokens = re.sub(r"[^a-z0-9]+", " ", sent_str.lower()).split()
    sentences_ted.append(tokens)

Two sample processed sentences:

In [64]:
len(sentences_ted)

266694

In [65]:
print(sentences_ted[0])
print(sentences_ted[1])

['here', 'are', 'two', 'reasons', 'companies', 'fail', 'they', 'only', 'do', 'more', 'of', 'the', 'same', 'or', 'they', 'only', 'do', 'what', 's', 'new']
['to', 'me', 'the', 'real', 'real', 'solution', 'to', 'quality', 'growth', 'is', 'figuring', 'out', 'the', 'balance', 'between', 'two', 'activities', 'exploration', 'and', 'exploitation']


### Part 2: Word Frequencies

If you store the counts of the top 1000 words in a list called `counts_ted_top1000`, the code below will plot the histogram requested in the writeup.

In [66]:
import collections

In [67]:
c = collections.Counter()
for sentence in sentences_ted:
    for word in sentence:
        c[word] += 1
        
#print(c.most_common)

words_top_ted = list(w[0] for w in c.most_common(1000))
counts_top_ted = list(w[1] for w in c.most_common(1000))

#print(words_top_ted)

Plot distribution of top-1000 words

In [ ]:
hist, edges = np.histogram(words_top_ted, density=True, bins=100, normed=True)

p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="Top-1000 words distribution")
p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], line_color="#555555")
show(p)

### Part 3: Train Word2Vec

In [68]:
from gensim.models import Word2Vec

In [69]:
model_ted = Word2Vec(sentences_ted, size=100, window=5, min_count=5, workers=4)
model_ted.save("ted_model")
model_ted = Word2Vec.load("ted_model")

### Part 4: Ted Learnt Representations

Finding similar words: (see gensim docs for more functionality of `most_similar`)

In [70]:
model_ted.most_similar("man")

[('woman', 0.8623941540718079),
 ('guy', 0.8020063042640686),
 ('lady', 0.7517378330230713),
 ('girl', 0.7424250245094299),
 ('boy', 0.7260259389877319),
 ('gentleman', 0.7158139944076538),
 ('soldier', 0.7101040482521057),
 ('kid', 0.684889554977417),
 ('poet', 0.6725330352783203),
 ('surgeon', 0.6574485898017883)]

In [71]:
model_ted.most_similar("computer")

[('machine', 0.7463439702987671),
 ('software', 0.7283808588981628),
 ('robot', 0.7065974473953247),
 ('device', 0.6993850469589233),
 ('3d', 0.674721360206604),
 ('program', 0.661706805229187),
 ('desktop', 0.6491623520851135),
 ('printer', 0.6485422849655151),
 ('chip', 0.6469804048538208),
 ('video', 0.6443352699279785)]

In [72]:
model_ted.most_similar("solution")

[('strategy', 0.6890659332275391),
 ('problem', 0.6789311170578003),
 ('challenge', 0.6711745858192444),
 ('issue', 0.6573665738105774),
 ('approach', 0.6467491388320923),
 ('concept', 0.6324085593223572),
 ('response', 0.6308174729347229),
 ('tool', 0.6259669065475464),
 ('task', 0.6233576536178589),
 ('mechanism', 0.6106288433074951)]

In [73]:
model_ted.most_similar(positive=['woman', 'king'], negative=['man'])

[('president', 0.7985718846321106),
 ('poet', 0.7724171876907349),
 ('named', 0.7710288763046265),
 ('queen', 0.7601550221443176),
 ('martin', 0.7555313110351562),
 ('james', 0.7550230026245117),
 ('john', 0.7460649609565735),
 ('barack', 0.7429082989692688),
 ('obama', 0.7402222156524658),
 ('luther', 0.7393237948417664)]

#### t-SNE visualization
To use the t-SNE code below, first put a list of the top 1000 words (as strings) into a variable `words_top_ted`. The following code gets the corresponding vectors from the model, assuming it's called `model_ted`:

In [74]:
# This assumes words_top_ted is a list of strings, the top 1000 words
words_top_vec_ted = model_ted[words_top_ted]

In [75]:
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2, random_state=0)
words_top_ted_tsne = tsne.fit_transform(words_top_vec_ted)

In [76]:
p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="word2vec T-SNE for most common words")

source = ColumnDataSource(data=dict(x1=words_top_ted_tsne[:,0],
                                    x2=words_top_ted_tsne[:,1],
                                    names=words_top_ted))

p.scatter(x="x1", y="x2", size=8, source=source)

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

show(p)

### Part 5: Wiki Learnt Representations

Download dataset

In [77]:
if not os.path.isfile('wikitext-103-raw-v1.zip'):
    urllib.request.urlretrieve("https://s3.amazonaws.com/research.metamind.io/wikitext/wikitext-103-raw-v1.zip", filename="wikitext-103-raw-v1.zip")

In [78]:
with zipfile.ZipFile('wikitext-103-raw-v1.zip', 'r') as z:
    input_text = str(z.open('wikitext-103-raw/wiki.train.raw', 'r').read(), encoding='utf-8') # Thanks Robert Bastian

Preprocess sentences (note that it's important to remove small sentences for performance)

In [79]:
sentences_wiki = []
for line in input_text.split('\n'):
    s = [x for x in line.split('.') if x and len(x.split()) >= 5]
    sentences_wiki.extend(s)
    
for s_i in range(len(sentences_wiki)):
    sentences_wiki[s_i] = re.sub("[^a-z]", " ", sentences_wiki[s_i].lower())
    sentences_wiki[s_i] = re.sub(r'\([^)]*\)', '', sentences_wiki[s_i])
del input_text

In [80]:
# sample 1/5 of the data
shuffle(sentences_wiki)
print(len(sentences_wiki))
sentences_wiki = sentences_wiki[:int(len(sentences_wiki)/5)]
print(len(sentences_wiki))

4267112
853422


In [81]:
sentences_wiki_2=[]
for wr in sentences_wiki:
    sentences_wiki_2.append(wr.split())
#print(sentences_wiki_2[0])

In [82]:
wc=collections.Counter()
for s in sentences_wiki:
    for word in s.split():
        wc[word]+=1
words_top_wiki=list(k[0] for k in wc.most_common(1000))
counts_wiki_top1000=list(k[1] for k in wc.most_common(1000))
print(words_top_wiki[:10])

['the', 'of', 'and', 'in', 'to', 'a', 'was', 's', 'on', 'as']


Now, repeat all the same steps that you performed above. You should be able to reuse essentially all the code.

In [83]:
c_2 = collections.Counter()
for sentence in sentences_wiki:
    for word in sentence.split():
        c_2[word] += 1

words_top_wiki = list(wrd[0] for wrd in c_2.most_common(1000))
counts_top1000_wiki = list(wrd[1] for wrd in c_2.most_common(1000))
print(counts_top1000_wiki[:10])

[1285892, 546874, 500091, 435571, 398313, 357465, 214287, 164963, 153838, 144792]


In [ ]:
hist, edges = np.histogram(words_top_wiki, density=True, bins=100, normed=True)

p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="Top-1000 words distribution")
p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], line_color="#555555")
show(p)

In [84]:
wiki_model = Word2Vec(sentences_wiki_2, size=100, window=5, min_count=5, workers=4)
wiki_model.save("wiki_model")
model_wiki = Word2Vec.load("wiki_model")

In [85]:
model_wiki.most_similar("man")

[('woman', 0.7012144923210144),
 ('girl', 0.6198760271072388),
 ('boy', 0.6102288961410522),
 ('person', 0.6069545745849609),
 ('soldier', 0.5723516345024109),
 ('men', 0.5614178776741028),
 ('kid', 0.5516582131385803),
 ('someone', 0.5452362298965454),
 ('creature', 0.5344418287277222),
 ('dog', 0.5288004279136658)]

In [86]:
model_wiki.most_similar("computer")

[('software', 0.7816450595855713),
 ('computers', 0.722388505935669),
 ('hardware', 0.7007551789283752),
 ('user', 0.6909362077713013),
 ('handheld', 0.6895475387573242),
 ('controller', 0.6881362795829773),
 ('interface', 0.6866271495819092),
 ('interactive', 0.6788220405578613),
 ('graphics', 0.670163631439209),
 ('graphical', 0.669544517993927)]

In [87]:
model_wiki.most_similar("solution")

[('problem', 0.7181879281997681),
 ('function', 0.7173067331314087),
 ('remedy', 0.7102504968643188),
 ('mechanism', 0.6947425007820129),
 ('object', 0.6839619278907776),
 ('molecule', 0.6784687042236328),
 ('method', 0.6779582500457764),
 ('measure', 0.6750925183296204),
 ('enzyme', 0.6570701599121094),
 ('solutions', 0.6569666862487793)]

In [88]:
model_wiki.most_similar(positive=['woman', 'king'], negative=['man'])

[('queen', 0.7343854308128357),
 ('monarch', 0.6997648477554321),
 ('empress', 0.6934734582901001),
 ('throne', 0.6504689455032349),
 ('elizabeth', 0.644758939743042),
 ('mary', 0.6372894644737244),
 ('prince', 0.6365466117858887),
 ('emperor', 0.6298166513442993),
 ('isabella', 0.6141037344932556),
 ('dowager', 0.6120776534080505)]

#### t-SNE visualization

In [89]:
# This assumes words_top_wiki is a list of strings, the top 1000 words
words_top_vec_wiki = model_wiki[words_top_wiki]

tsne = TSNE(n_components=2, random_state=0)
words_top_wiki_tsne = tsne.fit_transform(words_top_vec_wiki)

In [90]:
p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="word2vec T-SNE for most common words")

source = ColumnDataSource(data=dict(x1=words_top_wiki_tsne[:,0],
                                    x2=words_top_wiki_tsne[:,1],
                                    names=words_top_wiki))

p.scatter(x="x1", y="x2", size=8, source=source)

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

show(p)